In [1]:
import pandas as pd

import panel as pn  # GUI
pn.extension()
import panel as pn
import param

import os
import openai

import csv

from dotenv import load_dotenv, find_dotenv

from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory


import requests
from pydantic.v1 import BaseModel, Field
import datetime

import wikipedia
import csv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [3]:
# Define the input schema
class GarmentDescription(BaseModel):
    name: str = Field(..., description="Name of the item of clothing")
    description: str = Field(..., description="Description of the item of clothing")
@tool(args_schema=GarmentDescription)
def add_item_to_db(name: str, description: str) -> None:
    """This function saves the item of clothing inside the database."""
    with open('data.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, description])
    return None

In [4]:
@tool
def extract_all_items_from_db() -> str:
    """This function extracts all information from the database in csv format."""
    # Read the CSV file as a single string
    with open('data.csv', 'r', newline='', encoding='utf-8') as csvfile:
        content = csvfile.read()  # Reads the entire file as a single string
    return content

In [5]:
tools = [get_current_temperature, add_item_to_db, extract_all_items_from_db]

In [8]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

system_promt = "You are a fashion assistant and you are helping a user to choose items from their wardrobe.\
      The user can ask you to add an item to the database, or to provide a list of all items in the database.\
      The user can also ask you to suggest an outfit for today in a given location, \
    in this case you should first look up the current temperature at the location, extract items from their wardrobe and make fashinable suggestion from existing\
        items. Don't include items, if they are not in the database!"

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helping a user to choose items from their wardrobe. The user can ask you to add an item to the database, or to provide a list of all items in the database. You can also provide the current temperature at a given location."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        print(result)
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=700)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [9]:
cb = cbfs(tools)

inp = pn.widgets.TextInput(placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=1000),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# WearAI')),
    pn.Tabs(('Conversation', tab1))
)
dashboard.show()   

Launching server at http://localhost:64879


{'input': 'Hey!', 'chat_history': [HumanMessage(content='Hey!'), AIMessage(content='Hello! How can I assist you today?')], 'output': 'Hello! How can I assist you today?'}
{'input': 'What do I have in my wardrobe?', 'chat_history': [HumanMessage(content='Hey!'), AIMessage(content='Hello! How can I assist you today?'), HumanMessage(content='What do I have in my wardrobe?'), AIMessage(content='Here is a list of items in your wardrobe:\n\n1. Sweater: Red light sweater with a white image of a deer\n2. Shoes: Blue summer strap sandals with sparkles\n3. Jacket: Black leather jacket with silver zippers and a belt\n4. Jeans: Classic blue skinny jeans with ripped knees\n5. Hat: Beige straw hat with a black ribbon around the brim\n6. Scarf: Long wool scarf in green and white stripes\n7. Shirt: White cotton shirt with short sleeves and a pocket\n8. Skirt: Pleated navy-blue skirt with a floral pattern\n9. Gloves: Thick black winter gloves with touchscreen tips\n10. Boots: Brown ankle boots with a l